In [4]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Path to your cloned repo folder
data_dir = "coe379L-sp25/datasets/unit03/Project3"  # or use the full path

# Image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Wrap in DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print classes
print("Classes:", dataset.classes)


Classes: ['damage', 'no_damage']


In [7]:
from PIL import Image
from collections import Counter

# Count total images per class
for cls in dataset.classes:
    folder = os.path.join(data_dir, cls)
    print(f"{cls}: {len(os.listdir(folder))} images")

# Check size of a sample image
sample_path = os.path.join(data_dir, dataset.classes[0], os.listdir(os.path.join(data_dir, dataset.classes[0]))[0])
img = Image.open(sample_path)
print(f"Sample image size: {img.size}")

shape_counter = Counter()

for cls in dataset.classes:
    folder = os.path.join(data_dir, cls)
    for img_name in os.listdir(folder):
        path = os.path.join(folder, img_name)
        try:
            with Image.open(path) as img:
                shape_counter[img.size] += 1
        except:
            pass  

print("Most common image shapes:")
for shape, count in shape_counter.most_common(5):
    print(f"{shape}: {count} images")


damage: 14170 images
no_damage: 7152 images
Sample image size: (128, 128)
Most common image shapes:
(128, 128): 21322 images


In [8]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import numpy as np

# Get all indices and labels
indices = list(range(len(dataset)))
labels = [sample[1] for sample in dataset.samples]

# Split: 80% train, 10% val, 10% test
train_idx, temp_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=42
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, stratify=[labels[i] for i in temp_idx], random_state=42
)

# Create subsets
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)
test_dataset = Subset(dataset, test_idx)


In [9]:
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Train set size: 17057
Validation set size: 2132
Test set size: 2133


In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class DenseANN(nn.Module):
    def __init__(self):
        super(DenseANN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 128 * 3, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss, and optimizer
model = DenseANN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ------------------------------
# 🔁 Training & Evaluation Functions
# ------------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

# ------------------------------
# 🚀 Run Training Loop
# ------------------------------
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  ✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"  🔍 Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

# ------------------------------
# 🧪 Final Test Accuracy
# ------------------------------
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\n🎯 Test Accuracy: {test_acc:.4f}")


Using device: cpu
Epoch 1/10
  ✅ Train Loss: 0.7417 | Acc: 0.7286
  🔍 Val   Loss: 0.5229 | Acc: 0.7575
Epoch 2/10
  ✅ Train Loss: 0.4831 | Acc: 0.7824
  🔍 Val   Loss: 0.4831 | Acc: 0.7641
Epoch 3/10
  ✅ Train Loss: 0.4489 | Acc: 0.7983
  🔍 Val   Loss: 0.5682 | Acc: 0.7265
Epoch 4/10
  ✅ Train Loss: 0.4324 | Acc: 0.8095
  🔍 Val   Loss: 0.7094 | Acc: 0.6698
Epoch 5/10
  ✅ Train Loss: 0.4305 | Acc: 0.8055
  🔍 Val   Loss: 0.4605 | Acc: 0.7885
Epoch 6/10
  ✅ Train Loss: 0.4033 | Acc: 0.8276
  🔍 Val   Loss: 0.4728 | Acc: 0.7767
Epoch 7/10
  ✅ Train Loss: 0.3861 | Acc: 0.8334
  🔍 Val   Loss: 0.4497 | Acc: 0.7922
Epoch 8/10
  ✅ Train Loss: 0.3689 | Acc: 0.8432
  🔍 Val   Loss: 0.6224 | Acc: 0.7087
Epoch 9/10
  ✅ Train Loss: 0.3598 | Acc: 0.8458
  🔍 Val   Loss: 0.5000 | Acc: 0.7875
Epoch 10/10
  ✅ Train Loss: 0.3431 | Acc: 0.8568
  🔍 Val   Loss: 0.4762 | Acc: 0.7842

🎯 Test Accuracy: 0.8148


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------
# 🧠 LeNet-5 with Dropout
# ------------------------------
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)     # → (6, 124, 124)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)                        # → (6, 62, 62)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)   # → (16, 58, 58) → pool → (16, 29, 29)

        self.fc1 = nn.Linear(16 * 29 * 29, 120)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ------------------------------
# ⚙️ Setup
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# ------------------------------
# Training / Evaluation Functions
# ------------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

# ------------------------------
# 🧠 Early Stopping Logic
# ------------------------------
EPOCHS = 20
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  ✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"  🔍 Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_lenet5_model.pth")  # Save best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break

# ------------------------------
# 🧪 Load Best Model & Test
# ------------------------------
model.load_state_dict(torch.load("best_lenet5_model.pth"))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\n🎯 Final Test Accuracy: {test_acc:.4f}")


Epoch 1/20
  ✅ Train Loss: 0.3879 | Acc: 0.8261
  🔍 Val   Loss: 0.2225 | Acc: 0.9132
Epoch 2/20
  ✅ Train Loss: 0.2635 | Acc: 0.9035
  🔍 Val   Loss: 0.2081 | Acc: 0.9226
Epoch 3/20
  ✅ Train Loss: 0.2094 | Acc: 0.9239
  🔍 Val   Loss: 0.2219 | Acc: 0.9146
Epoch 4/20
  ✅ Train Loss: 0.1834 | Acc: 0.9296
  🔍 Val   Loss: 0.1924 | Acc: 0.9306
Epoch 5/20
  ✅ Train Loss: 0.1503 | Acc: 0.9414
  🔍 Val   Loss: 0.1697 | Acc: 0.9339
Epoch 6/20
  ✅ Train Loss: 0.1427 | Acc: 0.9458
  🔍 Val   Loss: 0.1838 | Acc: 0.9292
Epoch 7/20
  ✅ Train Loss: 0.1347 | Acc: 0.9496
  🔍 Val   Loss: 0.1892 | Acc: 0.9362
Epoch 8/20
  ✅ Train Loss: 0.1114 | Acc: 0.9581
  🔍 Val   Loss: 0.1510 | Acc: 0.9409
Epoch 9/20
  ✅ Train Loss: 0.1005 | Acc: 0.9617
  🔍 Val   Loss: 0.1808 | Acc: 0.9306
Epoch 10/20
  ✅ Train Loss: 0.0914 | Acc: 0.9639
  🔍 Val   Loss: 0.1629 | Acc: 0.9395
Epoch 11/20
  ✅ Train Loss: 0.0800 | Acc: 0.9707
  🔍 Val   Loss: 0.1640 | Acc: 0.9348
⏹️ Early stopping at epoch 11

🎯 Final Test Accuracy: 0.9498


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# ------------------------------
# 🧠 Alternate LeNet-5 CNN
# ------------------------------
class AlternateLeNet5(nn.Module):
    def __init__(self):
        super(AlternateLeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)     # (3,128,128) -> (6,128,128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)          # -> (6,64,64)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=2)    # -> (16,64,64)
        # pool again → (16,32,32)

        self.fc1 = nn.Linear(16 * 32 * 32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)  # Binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ------------------------------
# ⚙️ Setup
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlternateLeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# ------------------------------
# 🔁 Training / Evaluation Functions
# ------------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), total_correct / len(loader.dataset)

# ------------------------------
# 🚀 Training Loop
# ------------------------------
EPOCHS = 10  # You can increase to 20+ if you're not using early stopping
for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  ✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"  🔍 Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

# ------------------------------
# 🧪 Final Test Evaluation
# ------------------------------
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\n🎯 Final Test Accuracy: {test_acc:.4f}")


Epoch 1/10
  ✅ Train Loss: 0.3393 | Acc: 0.8560
  🔍 Val   Loss: 0.3051 | Acc: 0.8780
Epoch 2/10
  ✅ Train Loss: 0.1993 | Acc: 0.9238
  🔍 Val   Loss: 0.1821 | Acc: 0.9259
Epoch 3/10
  ✅ Train Loss: 0.1448 | Acc: 0.9438
  🔍 Val   Loss: 0.1498 | Acc: 0.9414
Epoch 4/10
  ✅ Train Loss: 0.1110 | Acc: 0.9597
  🔍 Val   Loss: 0.1453 | Acc: 0.9442
Epoch 5/10
  ✅ Train Loss: 0.0787 | Acc: 0.9706
  🔍 Val   Loss: 0.1527 | Acc: 0.9423
Epoch 6/10
  ✅ Train Loss: 0.0542 | Acc: 0.9818
  🔍 Val   Loss: 0.2332 | Acc: 0.9353
Epoch 7/10
  ✅ Train Loss: 0.0490 | Acc: 0.9832
  🔍 Val   Loss: 0.1642 | Acc: 0.9423
Epoch 8/10
  ✅ Train Loss: 0.0340 | Acc: 0.9889
  🔍 Val   Loss: 0.2342 | Acc: 0.9315
Epoch 9/10
  ✅ Train Loss: 0.0500 | Acc: 0.9826
  🔍 Val   Loss: 0.2443 | Acc: 0.9329
Epoch 10/10
  ✅ Train Loss: 0.0217 | Acc: 0.9928
  🔍 Val   Loss: 0.2257 | Acc: 0.9456

🎯 Final Test Accuracy: 0.9466


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# ------------------------------
# 🧠 Define Optimized Alternate LeNet-5
# ------------------------------
class OptimizedAlternateLeNet5(nn.Module):
    def __init__(self):
        super(OptimizedAlternateLeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=2)

        self.fc1 = nn.Linear(16 * 32 * 32, 120)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# ------------------------------
# 🔁 Data Augmentation & Reload Data
# ------------------------------
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Reload dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split
indices = list(range(len(dataset)))
labels = [sample[1] for sample in dataset.samples]

train_idx, temp_idx = train_test_split(indices, test_size=0.2, stratify=labels, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=[labels[i] for i in temp_idx], random_state=42)

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)
test_dataset = Subset(dataset, test_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ------------------------------
# ⚙️ Training Setup
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OptimizedAlternateLeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# ------------------------------
# 🔁 Training & Evaluation Functions
# ------------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct = 0.0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct = 0.0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

# ------------------------------
# 🚀 Training Loop with Early Stopping
# ------------------------------
EPOCHS = 30
best_val_loss = float('inf')
patience = 4
counter = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  ✅ Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"  🔍 Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "optimized_lenet5.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break

# ------------------------------
# 🧪 Final Test Accuracy
# ------------------------------
model.load_state_dict(torch.load("optimized_lenet5.pth"))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\n🎯 Final Test Accuracy: {test_acc:.4f}")


Epoch 1/30
  ✅ Train Loss: 0.3880 | Acc: 0.8304
  🔍 Val   Loss: 0.2317 | Acc: 0.9099
Epoch 2/30
  ✅ Train Loss: 0.2581 | Acc: 0.9085
  🔍 Val   Loss: 0.3018 | Acc: 0.8912
Epoch 3/30
  ✅ Train Loss: 0.2211 | Acc: 0.9170
  🔍 Val   Loss: 0.1718 | Acc: 0.9334
Epoch 4/30
  ✅ Train Loss: 0.2011 | Acc: 0.9244
  🔍 Val   Loss: 0.1835 | Acc: 0.9296
Epoch 5/30
  ✅ Train Loss: 0.1910 | Acc: 0.9282
  🔍 Val   Loss: 0.1518 | Acc: 0.9423
Epoch 6/30
  ✅ Train Loss: 0.1819 | Acc: 0.9305
  🔍 Val   Loss: 0.1836 | Acc: 0.9231
Epoch 7/30
  ✅ Train Loss: 0.1707 | Acc: 0.9362
  🔍 Val   Loss: 0.1390 | Acc: 0.9447
Epoch 8/30
  ✅ Train Loss: 0.1658 | Acc: 0.9368
  🔍 Val   Loss: 0.1365 | Acc: 0.9456
Epoch 9/30
  ✅ Train Loss: 0.1602 | Acc: 0.9391
  🔍 Val   Loss: 0.1749 | Acc: 0.9226
Epoch 10/30
  ✅ Train Loss: 0.1554 | Acc: 0.9401
  🔍 Val   Loss: 0.1347 | Acc: 0.9465
Epoch 11/30
  ✅ Train Loss: 0.1387 | Acc: 0.9449
  🔍 Val   Loss: 0.1503 | Acc: 0.9447
Epoch 12/30
  ✅ Train Loss: 0.1406 | Acc: 0.9453
  🔍 Val   Loss

In [18]:
torch.save(model.state_dict(), "optimized_lenet5.pth")